### 🟩 CELL 1 — Mount Google Drive

In [1]:
# ===============================
# CELL 1: MOUNT GOOGLE DRIVE
# ===============================
# This allows Colab to access files from your Google Drive

from google.colab import drive
drive.mount('/content/drive')

# After mounting:
# Your files will be available at:
# /content/drive/MyDrive/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 🟩 CELL 2 — Import Required Libraries


In [2]:
# ===============================
# CELL 2: IMPORT LIBRARIES
# ===============================

import tensorflow as tf
from tensorflow.keras import layers, models


### 🟩 CELL 3 — Define Dataset Path

In [3]:
# ===============================
# CELL 3: DATASET PATH
# ===============================

dataset_path = "/content/drive/MyDrive/cats and dogs"


### 🟩 CELL 4 — Load Images + Split into Train & Validation

In [4]:
# ===============================
# CELL 4: LOAD DATASET
# ===============================
# image_dataset_from_directory does ALL of this:
# - Reads images
# - Resizes them
# - Converts them to tensors
# - Assigns labels
# - Creates batches
# - Splits into training & validation

train_data = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,        # 20% data for validation
    subset="training",           # training data
    seed=123,                    # ensures same split every time
    image_size=(128, 128),       # resize all images
    batch_size=32
)

val_data = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",         # validation data
    seed=123,
    image_size=(128, 128),
    batch_size=32
)


Found 1000 files belonging to 2 classes.
Using 800 files for training.
Found 1000 files belonging to 2 classes.
Using 200 files for validation.


### 🟩 CELL 5 — Check Class Labels

In [5]:
# ===============================
# CELL 5: CHECK CLASS NAMES
# ===============================

class_names = train_data.class_names
print(class_names)

# Example output:
# ['cats', 'dogs']
#
# cats  -> label 0
# dogs  -> label 1


['cats_set', 'dogs_set']


### 🟩 CELL 6 — Normalize Images

In [6]:
# ===============================
# CELL 6: NORMALIZATION
# ===============================
# This layer converts pixel values from:
# [0, 255] → [0, 1]

normalization_layer = layers.Rescaling(1./255)

# Apply normalization to each batch
train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))


### 🟩 CELL 7 — Build CNN Model

In [7]:
# ===============================
# CELL 7: BUILD CNN MODEL
# ===============================

model = models.Sequential([

    # --------------------------------
    # INPUT + CONVOLUTION LAYER 1
    # --------------------------------
    # - Image enters here
    # - 32 kernels of size 3x3 slide over image
    # - ReLU applied after convolution

    layers.Conv2D(
        filters=32,
        kernel_size=(3,3),
        activation='relu',
        input_shape=(128, 128, 3)  # RGB image
    ),

    # --------------------------------
    # POOLING LAYER 1
    # --------------------------------
    # - Reduces spatial size by half
    # - Keeps important features

    layers.MaxPooling2D(pool_size=(2,2)),

    # --------------------------------
    # CONVOLUTION LAYER 2
    # --------------------------------
    # - Learns more complex features
    # - Works on feature maps, not raw pixels

    layers.Conv2D(64, (3,3), activation='relu'),

    # --------------------------------
    # POOLING LAYER 2
    # --------------------------------

    layers.MaxPooling2D(pool_size=(2,2)),

    # --------------------------------
    # FLATTEN LAYER
    # --------------------------------
    # - Converts 2D feature maps → 1D vector
    # - Bridge between CNN and ANN

    layers.Flatten(),

    # --------------------------------
    # DENSE (FULLY CONNECTED) LAYER
    # --------------------------------
    # - Learns combinations of features

    layers.Dense(64, activation='relu'),

    # --------------------------------
    # OUTPUT LAYER
    # --------------------------------
    # - 2 neurons = 2 classes
    # - Softmax outputs probabilities

    layers.Dense(2, activation='softmax')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### 🟩 CELL 8 — Compile Model

In [8]:
# ===============================
# CELL 8: COMPILE MODEL
# ===============================

model.compile(
    optimizer='adam',                       # updates weights
    loss='sparse_categorical_crossentropy', # for integer labels
    metrics=['accuracy']                    # measure performance
)


### 🟩 CELL 9 — Train the Model

In [9]:
# ===============================
# CELL 9: TRAIN MODEL
# ===============================

history = model.fit(
    train_data,
    epochs=5,                 # number of full passes over data
    validation_data=val_data
)


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 135s 5s/step - accuracy: 0.4926 - loss: 0.8659 - val_accuracy: 0.4750 - val_loss: 0.6921
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.6179 - loss: 0.6622 - val_accuracy: 0.4550 - val_loss: 0.7264
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 941ms/step - accuracy: 0.6672 - loss: 0.5940 - val_accuracy: 0.6250 - val_loss: 0.6980
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 945ms/step - accuracy: 0.8338 - loss: 0.3864 - val_accuracy: 0.6750 - val_loss: 0.6468
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 926ms/step - accuracy: 0.9064 - loss: 0.2580 - val_accuracy: 0.6650 - val_loss: 0.8215


### 🟩 CELL 10 — Check Accuracy on Validation Data

In [10]:
# ===============================
# CELL 10: EVALUATE MODEL
# ===============================

model.evaluate(val_data)

# Output example:
# loss: 0.3
# accuracy: 0.90


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 225ms/step - accuracy: 0.6711 - loss: 0.8287


[0.8214801549911499, 0.6650000214576721]